In [35]:
import os
import numpy as np
import pandas as pd
import nltk
import re
import copy
from nltk import Tree
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer  
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [212]:
#select_type = ['NN', 'NNS', 'JJ', 'VB', 'VBD', 'VBP', 'VBZ', 'VBN', 'VBG', 'JJR', 'JJS']
select_type = ['JJ','JJR', 'JJS' ]
letter_filter = re.compile('[^a-zA-Z]')
snowball_stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
useless_word = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',\
                  'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',\
                  'z', 'as', 'an', 'and', 'are', 'were', 'at', 'by', 'down', 'for', 'from',\
                  'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out',\
                  'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through',\
                  'to', 'top', 'up', 'while', 'with', 'do'])

In [213]:
training_descrption_files = [os.path.join('data/descriptions_train', filename) \
                             for filename in os.listdir('data/descriptions_train') 
                             if filename.endswith('.txt')]
training_tag_files = [os.path.join('data/tags_train', filename)\
                      for filename in os.listdir('data/tags_train')\
                      if filename.endswith('.txt')]
test_descrption_files = [os.path.join('data/descriptions_test', filename) \
                             for filename in os.listdir('data/descriptions_test') 
                             if filename.endswith('.txt')]
training_descrption_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
training_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_descrption_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))

In [214]:
def remove_empty(a):
    return [i for i in a if len(i) > 0]

In [215]:
def find_type(s, type_select):
    dictionary = set()
    sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(s)]
    sentences = [[letter_filter.sub('', sentences[i][j].lower())\
              for j in range(len(sentences[i]))] for i in range(len(sentences))]
    sentences = [remove_empty(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    for sent in sentences:
        for e in sent:
            if e[1] in type_select:
                dictionary.add(snowball_stemmer.stem(e[0]))
    return dictionary

In [216]:
adj = []
for file in test_descrption_files:
    aaa = []
    sentences = [line for line in open(file, 'r+')]
    for sent in sentences:
        nvj = find_type(sent, select_type)
        for word in nvj:
            if word not in aaa and word not in useless_word:
                aaa.append(word)
    adj.append(aaa)

In [218]:
len(adj)

2000

In [153]:
def define_des_tag_vector():
    selected = {}
    for file in training_descrption_files:
        sentences = [line for line in open(file, 'r+')]
        for sent in sentences:
            nvj = find_type(sent, select_type)
            for word in nvj:
                if word in selected.keys():
                    selected[word] += 1
                else:
                    selected[word] = 1
    
    final = []
    for k in selected.keys():
        if selected[k] >= 2 and k not in final:
            final.append(k)
    
    final_copy = copy.deepcopy(final)
    for element in final:
        if element in useless_word:
            final_copy.remove(element)
    
    
    
    return final_copy

In [59]:
a = define_des_tag_vector()

In [60]:
len(a)

3995

In [61]:
train_tag_files = [os.path.join('data/tags_train', filename) \
                             for filename in os.listdir('data/tags_train') 
                             if filename.endswith('.txt')]
test_tag_files = [os.path.join('data/tags_test', filename) \
                             for filename in os.listdir('data/tags_test') 
                             if filename.endswith('.txt')]
train_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))
test_tag_files.sort(key=lambda x: int(x[x.rindex('/') + 1:-4]))

In [65]:
Y_training = []
for file in training_descrption_files:
    vector_1 = dict.fromkeys(a, 0)
    sentences = [line for line in open(file, 'r+')]
    for sent in sentences:
        for word in sent.split():
            i = letter_filter.sub('', word.lower())
            if i in a:
                vector_1[i] = 1
  
    mm = [vector_1[element] for element in a]
    Y_training.append(list(vector_1.values()))

In [66]:
Y_test = []
for file in test_descrption_files:
    vector_1 = dict.fromkeys(a, 0)
    sentences = [line for line in open(file, 'r+')]
    for sent in sentences:
        for word in sent.split():
            i = letter_filter.sub('', word.lower())
            if i in a:
                vector_1[i] = 1

    mm = [vector_1[element] for element in a]
    Y_test.append(list(vector_1.values()))

In [67]:
import pickle

pickle.dump( Y_training, open( "y_train_v6.p", "wb" ) )
pickle.dump( Y_test, open( "y_test_v6.p", "wb" ) )

In [219]:
pickle.dump( adj, open( "adj_list_test.p", "wb" ) )

In [ ]:
In this paper, we use different models and use a variety of data preprocessing methods to achieve